## Neural Networks 
- Constructed using the torch.nn package

### Typical training procedure for a neural network 
- Define the neural network that has some learnable params
- Iterate over a dataset of inputs 
- Process input through the network
- Conpute the loss 
- Propagate gradients back into the networks'params 
- Update the weights of the network, using update ruls: w = w - lr*gradient

let's follow

## Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3) # input, output, kernel size
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y= W*x + b
        self.fc1 = nn.Linear(16 * 6* 6, 120) #input, output features, 6*6 from image dims
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dims except the batch dims 
        num_features = 1
        for s in size:
            num_features *= s 
        return num_features

In [23]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [24]:
params = list(net.parameters())
print('total params:',len(params))
print(params[0].size())

total params: 10
torch.Size([6, 1, 3, 3])


In [25]:
for i in params:
    print(i.size())

torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [26]:
x = torch.randn(1, 1, 32, 32)
out = net(x)
print(out)

tensor([[ 0.0087,  0.0655,  0.0590, -0.0826, -0.1036, -0.0351,  0.1492,  0.1039,
          0.0212,  0.0232]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients

In [30]:
net.zero_grad()
out.backward(torch.randn(1, 10))

##  Loss function

In [59]:
output = net(x)
print('output: \n' ,output)
target = torch.randn(10)
target = target.view(1, -1)
print('target: \n' ,target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print('loss: \n',loss)

output: 
 tensor([[ 0.0087,  0.0655,  0.0590, -0.0826, -0.1036, -0.0351,  0.1492,  0.1039,
          0.0212,  0.0232]], grad_fn=<AddmmBackward>)
target: 
 tensor([[-0.3901,  1.2340, -0.8216,  0.4211, -0.1195,  1.1311, -0.2626, -1.1068,
         -1.2109, -0.8282]])
loss: 
 tensor(0.7792, grad_fn=<MseLossBackward>)


In [60]:
print(loss.grad_fn)

In [61]:
print(loss.grad_fn.next_functions[0][0])

In [62]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## Backprop

In [64]:
net.zero_grad() # zeros the gradient buffers of all params

In [65]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [66]:
loss.backward()

In [67]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0015,  0.0026, -0.0026,  0.0010,  0.0148,  0.0053])


## Update the weights 
- weight = weight - lr * gradients

In [68]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [69]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(x)
loss = criterion(output, target)
loss.backward()
optimizer.step()